# Проверка обученной модели для задачи мозаики

Этот ноутбук загружает сохраненную модель и проверяет её работу на нескольких примерах.

In [1]:
import os
import sys
from pathlib import Path
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

# Project root = parent of "federated_uncertainty_scripts"
ROOT = Path().resolve().parents[2] if Path().resolve().name == "regression" else Path().resolve().parents[1]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from federated_uncertainty.regression_uncertainty.datasets.mosaic_datasets import build_id_and_ood
from federated_uncertainty.regression_uncertainty.source.models.cnn import get_cnn
from federated_uncertainty.regression_uncertainty.source.models.resnet import get_resnet18
from federated_uncertainty.regression_uncertainty.source.models.utils import variance_link, natural_link, natural_to_gauss

print(f"ROOT: {ROOT}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

ModuleNotFoundError: No module named 'federated_uncertainty'

## Настройка путей и параметров

In [ ]:
# Путь к сохраненной модели
model_path = "./data/saved_models/regression/test/test1/mosaic_cnn_naturalFalse_seed0_arw0.0"

# Или укажите свой путь:
# model_path = "./results/mosaic_regression/mosaic_cnn_naturalFalse_seed0_arw0.0"

# Устройство
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Загружаем аргументы из сохраненного файла
args_file = os.path.join(model_path, "args.txt")
if os.path.exists(args_file):
    with open(args_file, "r") as f:
        args_dict = {}
        for line in f:
            if ":" in line:
                key, value = line.strip().split(":", 1)
                key = key.strip()
                value = value.strip()
                # Преобразуем типы
                if value.lower() == "true":
                    args_dict[key] = True
                elif value.lower() == "false":
                    args_dict[key] = False
                elif value.replace(".", "").replace("-", "").isdigit():
                    if "." in value:
                        args_dict[key] = float(value)
                    else:
                        args_dict[key] = int(value)
                else:
                    args_dict[key] = value
    
    print("Loaded arguments:")
    for key, value in args_dict.items():
        print(f"  {key}: {value}")
else:
    print(f"Warning: args.txt not found at {args_file}")
    print("Using default arguments...")
    args_dict = {
        "network": "cnn",
        "use_natural": False,
        "tile_size": 32,
        "data_root": "./data",
        "method_seed": 0,
    }

## Загрузка модели

In [ ]:
# Параметры модели
network_type = args_dict.get("network", "cnn")
tile_size = args_dict.get("tile_size", 32)
image_size = 2 * tile_size  # 64x64 для tile_size=32
use_natural = args_dict.get("use_natural", False)

# Создаем модель
model_kwargs = {
    'in_channels': 1,
    'image_size': image_size,
}

if network_type == "cnn":
    model = get_cnn(**model_kwargs)
elif network_type == "resnet":
    model = get_resnet18(**model_kwargs)
else:
    raise ValueError(f"Unknown network type: {network_type}")

model.to(device)
print(f"Model created: {network_type}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

# Загружаем веса
models_dir = os.path.join(model_path, "models")
if os.path.exists(models_dir):
    model_files = [f for f in os.listdir(models_dir) if f.endswith(".pt")]
    if model_files:
        # Загружаем первую модель (или можно загрузить все для ансамбля)
        model_file = sorted(model_files)[0]
        model_path_full = os.path.join(models_dir, model_file)
        model.load_state_dict(torch.load(model_path_full, map_location=device))
        print(f"Model loaded from: {model_path_full}")
    else:
        print(f"No model files found in {models_dir}")
else:
    print(f"Models directory not found: {models_dir}")

model.eval()

## Загрузка данных

In [ ]:
# Загружаем тестовый датасет
data_root = args_dict.get("data_root", "./data")
method_seed = args_dict.get("method_seed", 0)

print("Loading test dataset...")
id_train, id_test, ood_fashion, ood_cifar10, ood_svhn, ood_mixture = build_id_and_ood(
    root=data_root,
    tile_size=tile_size,
    seed=method_seed,
    n_id_train=1000,  # Небольшое количество для быстрой проверки
    n_id_test=100,
    n_ood_each=100,
    download=True,
    normalize_images=True,
)

print(f"Test set size: {len(id_test)}")
print(f"Sample shape: {id_test[0][0].shape}")
print(f"Target range: [{id_test[0][1].item():.4f}, {id_test[-1][1].item():.4f}]")

## Предсказания на нескольких примерах

In [ ]:
# Берем несколько примеров
n_examples = 10
indices = list(range(min(n_examples, len(id_test))))

# Собираем данные
images = []
targets = []
for idx in indices:
    img, target = id_test[idx]
    images.append(img)
    targets.append(target)

images = torch.stack(images).to(device)
targets = torch.tensor(targets, dtype=torch.float32)

# Делаем предсказания
with torch.no_grad():
    outputs = model(images)
    
    # Преобразуем выходы в mean и variance
    if use_natural:
        mean, var = natural_to_gauss(*natural_link(outputs))
    else:
        mean, var = variance_link(outputs)
    
    predictions = mean.cpu().squeeze()
    uncertainties = var.cpu().squeeze()

# Денормализуем предсказания (если они были нормализованы в [0, 1])
# В оригинальном датасете normalize_target_to_unit=True, поэтому умножаем на 9999
targets_denorm = targets * 9999.0
predictions_denorm = predictions * 9999.0

print("\nПредсказания на примерах:")
print("-" * 80)
print(f"{'Index':<8} {'Target':<12} {'Prediction':<12} {'Error':<12} {'Uncertainty':<12}")
print("-" * 80)
for i in range(len(indices)):
    error = abs(targets_denorm[i].item() - predictions_denorm[i].item())
    print(f"{i:<8} {targets_denorm[i].item():<12.2f} {predictions_denorm[i].item():<12.2f} "
          f"{error:<12.2f} {uncertainties[i].item():<12.6f}")

# Вычисляем метрики
mse = torch.mean((targets - predictions) ** 2).item()
mae = torch.mean(torch.abs(targets - predictions)).item()
mse_denorm = torch.mean((targets_denorm - predictions_denorm) ** 2).item()
mae_denorm = torch.mean(torch.abs(targets_denorm - predictions_denorm)).item()

print("\n" + "=" * 80)
print("Метрики (нормализованные [0, 1]):")
print(f"  MSE: {mse:.6f}")
print(f"  MAE: {mae:.6f}")
print("\nМетрики (денормализованные [0, 9999]):")
print(f"  MSE: {mse_denorm:.2f}")
print(f"  MAE: {mae_denorm:.2f}")
print("=" * 80)

## Визуализация примеров

In [ ]:
# Визуализируем несколько примеров
n_show = min(6, len(indices))
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i in range(n_show):
    ax = axes[i]
    
    # Получаем изображение (денормализуем для визуализации)
    img = images[i].cpu().squeeze()
    
    # Если изображение нормализовано, нужно денормализовать для визуализации
    # Но для простоты показываем как есть
    ax.imshow(img, cmap='gray')
    ax.axis('off')
    
    target_val = targets_denorm[i].item()
    pred_val = predictions_denorm[i].item()
    error = abs(target_val - pred_val)
    uncertainty = uncertainties[i].item() * 9999.0  # Масштабируем неопределенность
    
    title = f"Target: {target_val:.0f}\nPred: {pred_val:.0f}\nError: {error:.0f}\nUnc: {uncertainty:.2f}"
    ax.set_title(title, fontsize=10)

plt.tight_layout()
plt.show()

## Оценка на всем тестовом наборе

In [ ]:
# Оцениваем на всем тестовом наборе
test_loader = DataLoader(id_test, batch_size=64, shuffle=False)

all_targets = []
all_predictions = []
all_uncertainties = []

model.eval()
with torch.no_grad():
    for images, targets in test_loader:
        images = images.to(device)
        
        outputs = model(images)
        
        if use_natural:
            mean, var = natural_to_gauss(*natural_link(outputs))
        else:
            mean, var = variance_link(outputs)
        
        all_targets.append(targets)
        all_predictions.append(mean.cpu().squeeze())
        all_uncertainties.append(var.cpu().squeeze())

all_targets = torch.cat(all_targets)
all_predictions = torch.cat(all_predictions)
all_uncertainties = torch.cat(all_uncertainties)

# Денормализуем
all_targets_denorm = all_targets * 9999.0
all_predictions_denorm = all_predictions * 9999.0

# Вычисляем метрики
mse_full = torch.mean((all_targets - all_predictions) ** 2).item()
mae_full = torch.mean(torch.abs(all_targets - all_predictions)).item()
mse_full_denorm = torch.mean((all_targets_denorm - all_predictions_denorm) ** 2).item()
mae_full_denorm = torch.mean(torch.abs(all_targets_denorm - all_predictions_denorm)).item()

print("=" * 80)
print("Оценка на всем тестовом наборе:")
print("=" * 80)
print(f"Размер тестового набора: {len(all_targets)}")
print("\nМетрики (нормализованные [0, 1]):")
print(f"  MSE: {mse_full:.6f}")
print(f"  MAE: {mae_full:.6f}")
print(f"  RMSE: {np.sqrt(mse_full):.6f}")
print("\nМетрики (денормализованные [0, 9999]):")
print(f"  MSE: {mse_full_denorm:.2f}")
print(f"  MAE: {mae_full_denorm:.2f}")
print(f"  RMSE: {np.sqrt(mse_full_denorm):.2f}")
print("\nСредняя неопределенность:")
print(f"  Mean uncertainty: {torch.mean(all_uncertainties).item():.6f}")
print(f"  Std uncertainty: {torch.std(all_uncertainties).item():.6f}")
print("=" * 80)

## График предсказаний vs реальных значений

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# График 1: Предсказания vs реальные значения
ax1 = axes[0]
ax1.scatter(all_targets_denorm.numpy(), all_predictions_denorm.numpy(), alpha=0.5, s=10)
ax1.plot([0, 9999], [0, 9999], 'r--', label='Ideal prediction')
ax1.set_xlabel('Real values')
ax1.set_ylabel('Predictions')
ax1.set_title('Predictions vs Real Values')
ax1.legend()
ax1.grid(True, alpha=0.3)

# График 2: Ошибки
ax2 = axes[1]
errors = (all_targets_denorm - all_predictions_denorm).abs().numpy()
ax2.hist(errors, bins=50, alpha=0.7, edgecolor='black')
ax2.set_xlabel('Absolute Error')
ax2.set_ylabel('Frequency')
ax2.set_title('Error Distribution')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Средняя абсолютная ошибка: {errors.mean():.2f}")
print(f"Медианная абсолютная ошибка: {np.median(errors):.2f}")
print(f"Максимальная ошибка: {errors.max():.2f}")